In [1]:

import os
from openai import OpenAI
import pandas as pd
import json
import duckdb
from pydantic import BaseModel, Field
from IPython.display import Markdown
from dotenv import load_dotenv


In [2]:
import logfire


In [3]:
# initialize the OpenAI client
load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")

client = OpenAI(api_key=openai_api_key)


logfire.configure()
logfire.instrument_openai(client)  

MODEL = "gpt-4o-mini"



Logfire project URL: ]8;id=325713;https://logfire-us.pydantic.dev/2500mbytes/starter-project\https://logfire-us.pydantic.dev/2500mbytes/starter-project]8;;\


In [4]:
# define the path to the transactional data
TRANSACTION_DATA_FILE_PATH = 'Store_Sales_Price_Elasticity_Promotions_Data.parquet'

In [5]:
# prompt template for step 2 of tool 1
SQL_GENERATION_PROMPT = """
Generate an SQL query based on a prompt. Do not reply with anything besides the SQL query.
The prompt is: {prompt}

The available columns are: {columns}
The table name is: {table_name}
"""

In [6]:
# code for step 2 of tool 1
def generate_sql_query(prompt: str, columns: list, table_name: str) -> str:
    """Generate an SQL query based on a prompt"""
    formatted_prompt = SQL_GENERATION_PROMPT.format(prompt=prompt, 
                                                    columns=columns, 
                                                    table_name=table_name)

    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": formatted_prompt}],
    )
    
    return response.choices[0].message.content

In [7]:
# code for tool 1
def lookup_sales_data(prompt: str) -> str:
    """Implementation of sales data lookup from parquet file using SQL"""
    try:

        # define the table name
        table_name = "sales"
        
        # step 1: read the parquet file into a DuckDB table
        df = pd.read_parquet(TRANSACTION_DATA_FILE_PATH)
        duckdb.sql(f"CREATE TABLE IF NOT EXISTS {table_name} AS SELECT * FROM df")

        # step 2: generate the SQL code
        sql_query = generate_sql_query(prompt, df.columns, table_name)
        # clean the response to make sure it only includes the SQL code
        sql_query = sql_query.strip()
        sql_query = sql_query.replace("```sql", "").replace("```", "")
        
        # step 3: execute the SQL query
        result = duckdb.sql(sql_query).df()
        
        return result.to_string()
    except Exception as e:
        return f"Error accessing data: {str(e)}"

In [8]:
example_data = lookup_sales_data("Show me all the sales for store 1320 on November 1st, 2021")
print(example_data)

05:05:05.916 Chat Completion with 'gpt-4o-mini' [LLM]
    Store_Number  SKU_Coded  Product_Class_Code  Sold_Date  Qty_Sold  Total_Sale_Value  On_Promo
0           1320    6173050               22875 2021-11-01         1          4.990000         0
1           1320    6174250               22875 2021-11-01         1          0.890000         0
2           1320    6176200               22975 2021-11-01         2         99.980003         0
3           1320    6176800               22800 2021-11-01         1         14.970000         0
4           1320    6177250               22975 2021-11-01         1          6.890000         0
5           1320    6177300               22800 2021-11-01         1          9.990000         0
6           1320    6177350               22800 2021-11-01         2         16.980000         0
7           1320    6177700               22875 2021-11-01         1          3.190000         0
8           1320    6178000               22875 2021-11-01         2     

In [9]:
# Construct prompt based on analysis type and data subset
DATA_ANALYSIS_PROMPT = """
Analyze the following data: {data}
Your job is to answer the following question: {prompt}
"""

In [10]:
# code for tool 2
def analyze_sales_data(prompt: str, data: str) -> str:
    """Implementation of AI-powered sales data analysis"""
    formatted_prompt = DATA_ANALYSIS_PROMPT.format(data=data, prompt=prompt)

    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": formatted_prompt}],
    )
    
    analysis = response.choices[0].message.content
    return analysis if analysis else "No analysis could be generated"

In [11]:
print(analyze_sales_data(prompt="what trends do you see in this data", 
                         data=example_data))

05:05:06.709 Chat Completion with 'gpt-4o-mini' [LLM]
Analyzing the provided sales data from Store 1320 for the date November 1, 2021, several trends and observations emerge:

1. **Product Diversity**: The dataset contains sales for multiple product classes (e.g., 22875, 22800, 22975, 24400), indicating a diverse range of products offered by the store. This diversity could appeal to different customer preferences.

2. **Sales Volume**: The quantities sold (Qty_Sold) range from 1 to 5. Notably, some products sold in higher quantities (e.g., SKU 6190050 sold 5 units), indicating that certain items are more popular than others. This can suggest particular trends or customer preferences regarding which products are in higher demand.

3. **Total Sales Value Variation**: The `Total_Sale_Value` varies significantly among products. Some high-value items (e.g., SKU 6176200 with a total sale of around $99.98) can contribute more to overall sales revenue, while other products have lower sale valu

In [12]:
# Define tools/functions that can be called by the model
tools = [
    {
        "type": "function",
        "function": {
            "name": "lookup_sales_data",
            "description": "Look up data from Store Sales Price Elasticity Promotions dataset",
            "parameters": {
                "type": "object",
                "properties": {
                    "prompt": {"type": "string", "description": "The unchanged prompt that the user provided."}
                },
                "required": ["prompt"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "analyze_sales_data", 
            "description": "Analyze sales data to extract insights",
            "parameters": {
                "type": "object",
                "properties": {
                    "data": {"type": "string", "description": "The lookup_sales_data tool's output."},
                    "prompt": {"type": "string", "description": "The unchanged prompt that the user provided."}
                },
                "required": ["data", "prompt"]
            }
        }
    }
]

# Dictionary mapping function names to their implementations
tool_implementations = {
    "lookup_sales_data": lookup_sales_data,
    "analyze_sales_data": analyze_sales_data
}

In [13]:
# code for executing the tools returned in the model's response
def handle_tool_calls(tool_calls, messages):

    for tool_call in tool_calls:   
        function = tool_implementations[tool_call.function.name]
        function_args = json.loads(tool_call.function.arguments)
        result = function(**function_args)
        messages.append({"role": "tool", "content": result, "tool_call_id": tool_call.id})
        
    return messages

In [14]:
SYSTEM_PROMPT = """
You are a helpful assistant that can answer questions about the Store Sales Price Elasticity Promotions dataset.
"""

In [15]:
def run_agent(messages):
    print("Running agent with messages:", messages)

    if isinstance(messages, str):
        messages = [{"role": "user", "content": messages}]
        
    # Check and add system prompt if needed
    if not any(
            isinstance(message, dict) and message.get("role") == "system" for message in messages
        ):
            system_prompt = {"role": "system", "content": SYSTEM_PROMPT}
            messages.append(system_prompt)

    while True:
        print("Making router call to OpenAI")
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools,
        )
        messages.append(response.choices[0].message)
        tool_calls = response.choices[0].message.tool_calls
        print("Received response with tool calls:", bool(tool_calls))

        # if the model decides to call function(s), call handle_tool_calls
        if tool_calls:
            print("Processing tool calls")
            messages = handle_tool_calls(tool_calls, messages)
        else:
            print("No tool calls, returning final response")
            return response.choices[0].message.content

In [16]:
result = run_agent('show me sales for top-10 (by sales) stores')

Running agent with messages: show me sales for top-10 (by sales) stores
Making router call to OpenAI
05:05:17.173 Chat Completion with 'gpt-4o-mini' [LLM]
Received response with tool calls: True
Processing tool calls
05:05:17.894 Chat Completion with 'gpt-4o-mini' [LLM]
Making router call to OpenAI
05:05:18.979 Chat Completion with 'gpt-4o-mini' [LLM]
Received response with tool calls: False
No tool calls, returning final response


In [17]:
#print(result)
# you can also print a formatted version of the result
Markdown(result)

Here are the sales figures for the top 10 stores by total sales:

| Store Number | Total Sales     |
|--------------|------------------|
| 2970         | $836,341.33      |
| 3300         | $619,660.17      |
| 1320         | $592,832.07      |
| 1650         | $580,443.01      |
| 1210         | $508,393.77      |
| 1100         | $497,509.53      |
| 3080         | $495,458.24      |
| 2750         | $453,664.81      |
| 1540         | $427,777.43      |
| 880          | $420,302.09      |